In [1]:
# importing libraries

import tensorflow as tf
import numpy as np
import pandas as pd
import glob

In [15]:
# defining params
# W must be less than or equal to CorrW

W = 20
CorrW = 50 

M = 0
N = 0

MD = 0
SD = 0

In [16]:
stocks = np.load('prepared_data\\stocks.npy', allow_pickle=True)
markets = np.load('prepared_data\\markets.npy', allow_pickle=True)

print(stocks.shape)
print(markets.shape)

M = markets.shape[0]
N = stocks.shape[0]

(572,)
(51,)


In [28]:
def build_corr_list(stock, market, CorrW):
    market_corr = []
    send = stock.shape[0] 
    mend = market.shape[0]
    while (mend >= CorrW) and (send >= CorrW):
        s_window = stock[send - CorrW : send]
        m_window = market[mend - CorrW : mend]
        corr_matrix = np.corrcoef(s_window, m_window)
        corr = corr_matrix[0, 1]
        market_corr.append(corr)
        send -= 1
        mend -= 1
    return market_corr[::-1]

def trim_2d_list(lst):
    min_length = min([len(x) for x in lst])
    for i in range(len(lst)):
        lst[i] = lst[i][-min_length:]

def get_market_windows(market, W, X):
    market_windows = []
    for i in range(X):
        market_window = market[i:i+W]
    return market_windows

In [27]:
all_data = []

for i in range(N):


    markets_corr = [] 
    for j in range(M):
        corr_list = build_corr_list(stocks[i][:, SD], markets[j][:, MD], CorrW)
        markets_corr.append(corr_list)

    trim_2d_list(markets_corr)

    X = len(markets_corr[0])

    trim_stock = stock[i][-X:]

    trim_markets = []
    for j in range(M):
        trim_markets.append(markets[j][-X:])


    break
    all_data.append(market_corr)

1119
1119
1119
1119
1119
1119
1119
1119
1119
1119
1119
1119
1119
1119
1119
1119
1119
1119
1119
1119
1119
1119
1119
1119
1119
1119
1119
1119
1119
1119
1119
1119
1119
1119
1119
1119
1119
1119
1119
1119
1119
1119
1119
1119
1119
1119
1119
1119
1119
1119
1119
